### Model Monitoring

In [ ]:
%store -r
import os,json
from dkube.sdk import *
from dkube.sdk.api import DkubeApi
from dkube.sdk.rsrcs import operator
from dkube.sdk.rsrcs.operator import DkubeCluster
from dkube.sdk.rsrcs import DkubeModelmonitor
from dkube.sdk.rsrcs.modelmonitor import DatasetClass,ModelType,DriftAlgo
from dkube.sdk.rsrcs.modelmonitor import DatasetFormat,DkubeModelmonitoralert

In [ ]:
api = DkubeApi(token=os.getenv("DKUBE_USER_ACCESS_TOKEN"))

#### Predict Dataset

In [ ]:
PREDICT_PREFIX = PREFIX +"/datacapture/"+endpoint_name+"/AllTraffic/"

In [ ]:
pname =  MONITOR_NAME+'-predict'

if DATA_SOURCE == 'aws_s3':
    try:
        dataset = DkubeDataset(DKUBEUSERNAME, name=pname,remote=True)
        dataset.update_dataset_source('aws_s3')
        dataset.update_awss3_details(
            bucket=BUCKET,
            prefix=PREDICT_PREFIX,key=os.getenv("AWS_ACCESS_KEY_ID",ACCESS_KEY),
            secret=os.getenv("AWS_SECRET_ACCESS_KEY",SECRET_KEY))
        api.create_dataset(dataset)
        
    except Exception as e:
        if e.reason:
            if e.reason.lower() != "conflict":
                response = e.body
                print(f"Failed[{response.code}]: {response.message}")
        else:
            raise e

#### Configure Sagemaker Cluster in Dkube

In [ ]:
pcluster = DkubeCluster(name=SAGEMAKER_DKUBE_CLUSTER_NAME)
pcluster.update_kind("sagemaker")
pcluster.update_access_keys(ACCESS_KEY,SECRET_KEY)
pcluster.cluster.access_keys.region = REGION_NAME

In [ ]:
api.configure_clusters(pcluster.cluster)

#### Import Deployment

In [ ]:
api.import_deployment(name=MONITOR_NAME,cluster=SAGEMAKER_DKUBE_CLUSTER_NAME,variant="sagemaker")

#### Model Monitor

In [ ]:
mm=DkubeModelmonitor(name=MONITOR_NAME)
mt=ModelType.Regression
dc_t=DatasetClass.Train

In [ ]:
text_file = open("transform-data.py", "r")
#read whole file to a string
script = text_file.read()
#close file
text_file.close()

with open('thresholds.json') as f:
    thresholds = json.load(f)

In [ ]:
mm.update_modelmonitor(model_type=str(mt),data_timezone="utc",input_data_type="tabular",thresholds=thresholds)

#### Training Details

In [ ]:
training_data = f'{DKUBE_BASE_DATASET}:'+ DKUBEUSERNAME
train_data_version = 'v1:'+api.get_dataset_versions(DKUBEUSERNAME,DKUBE_BASE_DATASET)[0]['version']['uuid']
prediction_data = MONITOR_NAME +'-predict:'+ DKUBEUSERNAME
labelled_data = MONITOR_NAME +'-groundtruth:'+ DKUBEUSERNAME
predict_data_format = str(DatasetFormat.Sagemakerlogs)

In [ ]:
mm.add_datasources(data_class=str(DatasetClass.Train),name=training_data,data_format=str(DatasetFormat.Tabular),version=train_data_version,transformer_script = script)
mm.add_datasources(data_class=str(DatasetClass.Predict),name=prediction_data,data_format=predict_data_format,date_suffix="yyyy/mm/dd/hh")
mm.add_datasources(data_class=str(DatasetClass.Labelled),name=labelled_data,data_format=str(DatasetFormat.Tabular),predict_col="charges",groundtruth_col="GT_target",timestamp_col="timestamp")

#### Add Drift monitoring details

In [ ]:
mm.update_drift_monitoring_details(enabled=True,frequency=5,algorithm='auto')

#### Create Model monitor

In [ ]:
api.modelmonitor_create(mm,wait_for_completion=True)

#### Extracting id of the Model Monitor

In [ ]:
id = api.modelmonitor_get_id(MONITOR_NAME)
id

#### Schema update

In [ ]:
api.modelmonitor_update_schema(id,label='charges',schema_class='continuous',schema_type="prediction_output",selected=False)
api.modelmonitor_update_schema(id,label='unique_id',schema_class='continuous',schema_type="row_id",selected=False)
api.modelmonitor_update_schema(id,label='timestamp',schema_class='continuous',schema_type="timestamp",selected=False)

## age and bmi to continuous
api.modelmonitor_update_schema(id,label='age',schema_class='continuous',schema_type='input_feature')
api.modelmonitor_update_schema(id,label='bmi',schema_class='continuous',schema_type='input_feature')

## select these features
api.modelmonitor_update_schema(id,label='sex',schema_type='input_feature',schema_class='categorical')
api.modelmonitor_update_schema(id,label='children',schema_type='input_feature',schema_class='categorical')
api.modelmonitor_update_schema(id,label='smoker',schema_type='input_feature',schema_class='categorical')
api.modelmonitor_update_schema(id,label='region',schema_type='input_feature',schema_class='categorical')

#### Performance Monitoring

In [ ]:
mm=DkubeModelmonitor(name=MONITOR_NAME)
mm.update_performance_monitoring_details(enabled=True,source_type="labelled_data",frequency=5)

In [ ]:
api.modelmonitor_update(id,mm)

#### Deployment monitoring

In [ ]:
mm=DkubeModelmonitor(name=MONITOR_NAME)
mm.update_deployment_monitoring_details(enabled=True,frequency=1,source_type="labelled_data",collect_metrics=True)

In [ ]:
api.modelmonitor_update(id,mm)

#### Add alerts

In [ ]:
alert = DkubeModelmonitoralert(name='age_alert')
alert.update_alert(alert_class = 'feature_drift',feature='age',threshold=0.02)
api.modelmonitor_add_alert(id,alert)

#### Start the model monitor

In [ ]:
api.modelmonitor_start(id)

#### Cleanup

In [ ]:
CLEANUP = False
if CLEANUP:
    from time import sleep
    RETRIES = 4
    while RETRIES:
        mm = api.modelmonitor_get(id)
        if mm["status"] and mm["status"]["state"].lower() != "active":
            break
        elif mm["status"] and mm["status"]["state"].lower() == "active":
            api.modelmonitor_stop(id)
        RETRIES -= 1
        sleep(5)
    else:
        raise TimeoutError("modelmonitor failed to stopped")
    api.modelmonitor_delete(id)
    